# **Spazzata di voltaggio**

Questo script esegue una spazzata lineare dal voltaggio `V0` al voltaggio `V1` con  `nV` punti. Per ogni valore l'$\texttt{ADC}$ esegue una misura con i seguenti parametri di campionamento:

* frequenza di sampling `fs`
* numero di acquisizioni `npt`

I valori risultanti di $\texttt{Ch1}$ e $\texttt{Ch2}$ vengono mediati e salvati.

In [13]:
import tdwf
import matplotlib
matplotlib.use('TkAgg')
import matplotlib.pyplot as plt 
import numpy as np
import time

V0 = -3
V1 = +3
nV = 101
npt = 101
fs  = 1e6

Vv = np.linspace(V0,V1,nV)

# -[Configurazione AD2]--------------------------------------------------------
#   1. Connessiene con AD2 e selezione configurazione 
ad2 = tdwf.AD2()
#   3. Configurazione generatore funzioni
wgen = tdwf.WaveGen(ad2.hdwf)
wgen.w1.func = tdwf.funcDC
wgen.w1.offs = V0
wgen.w1.start()
#   3. Configurazione oscilloscopio
scope = tdwf.Scope(ad2.hdwf)
scope.fs=fs
scope.npt=npt
scope.ch1.rng = 5
scope.ch2.rng = 5

# -[Ciclo di misura]-----------------------------------------------------------
fig, [ax1, ax2] = plt.subplots(1, 2, figsize=(12, 6))
fig.canvas.manager.set_window_title('Spazzata voltaggio')
ax2.yaxis.tick_right()
Ch1v = np.full((nV, 1), np.nan)
Ch2v = np.full((nV, 1), np.nan)

for ii, V in enumerate(Vv):
    wgen.w1.offs = V
    time.sleep(0.1)
    scope.sample()
    Ch1v[ii] = np.mean(scope.ch1.vals)
    Ch2v[ii] = np.mean(scope.ch2.vals)

    ax1.clear()    
    ax1.plot(Vv, Ch1v, ".", color="tab:orange", label="Ch1")
    ax1.plot(Vv, Ch2v, ".", color="tab:blue", label="Ch2")
    ax1.grid(True)
    ax1.set_xlabel("W1 [V]", fontsize=15)
    ax1.set_ylabel("Signals [V]", fontsize=15)
    ax1.legend()
    
    ax2.clear()    
    ax2.plot(Ch1v, Ch2v, ".", color="tab:orange")
    ax2.grid(True)
    ax2.set_xlabel("Ch1 [V]", fontsize=15)
    ax2.set_ylabel("Ch2 [V]", fontsize=15)
    ax2.yaxis.set_label_position('right')
    plt.tight_layout()
    plt.show(block=False)
    plt.pause(0.001)

ad2.close()

Dispositivo #1 [SN:210321B19D0C, hdwf=1] connesso!
Configurazione #1
Dispositivo disconnesso.
